In [6]:
!pip install implicit

In [1]:
!pip install openpyxl

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
#Import Statements

In [3]:
df=pd.read_excel('Online Retail.xlsx')

In [4]:
print(df.head())
print(df.info())
print(df.describe())


  InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

          InvoiceDate  UnitPrice  CustomerID         Country  
0 2010-12-01 08:26:00       2.55     17850.0  United Kingdom  
1 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  
2 2010-12-01 08:26:00       2.75     17850.0  United Kingdom  
3 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  
4 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       -----------

In [5]:
df.dropna(subset=['CustomerID'], inplace=True)


In [6]:
df.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

In [7]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df = df[df['Quantity'] > 0]
df = df[df['UnitPrice'] > 0]


In [8]:
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']


In [9]:
df.drop_duplicates(inplace=True)

df['CustomerID'].head()

0    17850.0
1    17850.0
2    17850.0
3    17850.0
4    17850.0
Name: CustomerID, dtype: float64

In [10]:
df_agg = df.groupby(['CustomerID', 'StockCode']).agg({'TotalPrice': 'sum'}).reset_index()

# Model Using The Collaborative filtering
User-Item

In [11]:
from sklearn.model_selection import train_test_split
from scipy.sparse import csr_matrix
from implicit.als import AlternatingLeastSquares

# Create a pivot table
user_item_matrix = df_agg.pivot(index='CustomerID', columns='StockCode', values='TotalPrice').fillna(0)

# Convert to sparse matrix
sparse_matrix = csr_matrix(user_item_matrix.values)

# Train-test split
train_data, test_data = train_test_split(user_item_matrix, test_size=0.2, random_state=42)

# Train ALS model
model = AlternatingLeastSquares(factors=20, regularization=0.1, iterations=50)
model.fit(sparse_matrix.T)



  0%|          | 0/50 [00:00<?, ?it/s]

# Model Using the Content Based Filtering


In [13]:
df['Description'] = df['Description'].fillna('')
sample_df = df.sample(frac=0.1, random_state=1)  # Use 10% of the data


In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

# Vectorize product descriptions
tfidf = TfidfVectorizer(stop_words='english',sublinear_tf=True)

tfidf_matrix = tfidf.fit_transform(sample_df['Description'])

# Compute similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

# Function to get recommendations
def get_recommendations(stock_code, cosine_sim=cosine_sim):
    idx = df[df['StockCode'] == stock_code].index[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    product_indices = [i[0] for i in sim_scores]
    return df.iloc[product_indices]

# Get recommendations for a specific StockCode
recommendations = get_recommendations('85123A')
print(recommendations[['StockCode', 'Description']])


      StockCode                       Description
3177      22386           JUMBO BAG PINK POLKADOT
6716     84849B            FAIRY SOAP SOAP HOLDER
8323      21802   CHRISTMAS TREE HEART DECORATION
8567      35954  SMALL FOLKART STAR CHRISTMAS DEC
8840      22834       HAND WARMER BABUSHKA DESIGN
9738      21700       BIG DOUGHNUT FRIDGE MAGNETS
11766     22592     CARDHOLDER HOLLY WREATH METAL
12146     21931            JUMBO STORAGE BAG SUKI
12813     22595           CHRISTMAS GINGHAM HEART
14006     22212        FOUR HOOK  WHITE LOVEBIRDS
